In [1]:
import psycopg2 as ps

In [2]:
conn = ps.connect(dbname="app",
                  user="postgres",
                  password="monografia",
                  host="mobapp_db_1")
cur = conn.cursor()

In [3]:
TABLEMETA = "dublin_v2"
TABLEPOINTS = "dublin_trajectories_v2"

In [4]:
cur.execute(f"CREATE TABLE {TABLEMETA} (trajectory_Id SERIAL PRIMARY KEY, line_id int, journey_id varchar,\
    time_frame timestamp, vehicle_journey_id int, operator varchar, vehicle_id int, trajectory_size numeric);")



In [5]:
create_sentence = f"CREATE TABLE {TABLEPOINTS} \
(id SERIAL PRIMARY KEY, trajectory_id int, instant timestamp, lat numeric,lng numeric, lat_mercator numeric, lng_mercator numeric, \
speed numeric, acceleration numeric, position numeric, delta_dist numeric, delta_time numeric, \
cum_dist int, cum_time int, bearing numeric, \
year int, month int, day int, hour int, minute int, stop_labels int, \
first_lat numeric, first_lng numeric, last_lat numeric, last_lng numeric, first_instant timestamp, \
last_instant timestamp, day_moment varchar, \
constraint fk_trajectories_dublin foreign key (trajectory_id) REFERENCES {TABLEMETA} (trajectory_id));"
print(create_sentence)

CREATE TABLE dublin_trajectories_v2 (id SERIAL PRIMARY KEY, trajectory_id int, instant timestamp, lat numeric,lng numeric, lat_mercator numeric, lng_mercator numeric, speed numeric, acceleration numeric, position numeric, delta_dist numeric, delta_time numeric, cum_dist int, cum_time int, bearing numeric, year int, month int, day int, hour int, minute int, stop_labels int, first_lat numeric, first_lng numeric, last_lat numeric, last_lng numeric, first_instant timestamp, last_instant timestamp, day_moment varchar, constraint fk_trajectories_dublin foreign key (trajectory_id) REFERENCES dublin_v2 (trajectory_id));


In [6]:
cur.execute(create_sentence)

In [7]:
conn.commit()

In [ ]:
ALTER TABLE dublin_v2
ADD first_lat numeric, 
ADD first_lng numeric, 
ADD last_lat numeric, 
ADD last_lng numeric, 
ADD first_instant timestamp, 
ADD last_instant timestamp ;

In [ ]:
UPDATE dublin_v2
SET first_lat = p.first_lat, first_lng = p.first_lng, last_lat = p.last_lat, last_lng = p.last_lng, 
first_instant = p.first_instant, last_instant = p.last_instant
FROM (
    select trajectory_id, first_lat, first_lng, last_lat, last_lng, first_instant, last_instant
    from dublin_trajectories_v2
    group by trajectory_id, first_lat, first_lng, last_lat, last_lng, first_instant, last_instant
) p
WHERE p.trajectory_id = dublin_v2.trajectory_id;